# Libraries

In [45]:
# install.packages('tm')
# install.packages("SnowballC")

Installing package into ‘/home/ggomarr/R/x86_64-pc-linux-gnu-library/3.3’
(as ‘lib’ is unspecified)


In [117]:
library(caTools)
library(ggplot2)
library(repr)
options(repr.plot.width=8,repr.plot.height=4)
library(SnowballC)
library(tm)
library(e1071)
library(rpart)
library(randomForest)

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin



# Read data

In [62]:
root_dir="~/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/"
work_dir="Course data/Part 7 - Natural Language Processing/Section 36 - Natural Language Processing"
setwd(paste(root_dir,work_dir,sep=''))
df=read.csv('Restaurant_Reviews.tsv',sep='\t',quote='',stringsAsFactors=FALSE)
# df=read.delim('Restaurant_Reviews.tsv',quote='',stringsAsFactors=FALSE)
head(df)

Review,Liked
Wow... Loved this place.,1
Crust is not good.,0
Not tasty and the texture was just nasty.,0
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
The selection on the menu was great and so were the prices.,1
Now I am getting angry and I want my damn pho.,0


In [93]:
reviewCorpus=VCorpus(VectorSource(df$Review))
writeLines(sapply(reviewCorpus[1:10],as.character))

Wow... Loved this place.
Crust is not good.
Not tasty and the texture was just nasty.
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.
The selection on the menu was great and so were the prices.
Now I am getting angry and I want my damn pho.
Honeslty it didn't taste THAT fresh.)
The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer.
The fries were great too.
A great touch.


In [94]:
processReview=function(review){
    review=gsub('[^[:alpha:] ]','',review)
#     review=removeNumbers(review)
#     review=removePunctuation(review)
    review=tolower(review)
    review=removeWords(review,stopwords("english"))
    review=stemDocument(review)
    review=stripWhitespace(review)
    review
}

In [95]:
reviewCorpus=tm_map(reviewCorpus,content_transformer(processReview))
writeLines(sapply(reviewCorpus[1:10],as.character))

wow love place
crust good
tasti textur just nasti
stop late may bank holiday rick steve recommend love
select menu great price
now get angri want damn pho
honeslti didnt tast fresh
potato like rubber tell made ahead time kept warmer
fri great
great touch


In [97]:
X=DocumentTermMatrix(reviewCorpus)
inspect(X)

<<DocumentTermMatrix (documents: 1000, terms: 1577)>>
Non-/sparse entries: 5435/1571565
Sparsity           : 100%
Maximal term length: 32
Weighting          : term frequency (tf)
Sample             :
     Terms
Docs  back food good great like place realli servic time will
  124    0    0    0     0    0     1      0      0    0    0
  133    0    1    1     0    0     0      0      0    0    0
  158    0    0    0     0    0     0      1      0    1    0
  237    0    0    0     0    0     0      0      0    0    0
  29     0    2    0     0    1     0      0      0    0    0
  43     0    0    1     0    0     0      0      0    0    0
  534    0    1    0     0    0     1      0      0    0    0
  716    0    0    0     1    0     0      0      0    0    0
  863    0    1    0     0    0     0      0      0    0    0
  885    0    0    0     0    0     0      0      0    1    0


In [98]:
X=removeSparseTerms(X,0.999)
inspect(X)

<<DocumentTermMatrix (documents: 1000, terms: 691)>>
Non-/sparse entries: 4549/686451
Sparsity           : 99%
Maximal term length: 12
Weighting          : term frequency (tf)
Sample             :
     Terms
Docs  back food good great like place realli servic time will
  124    0    0    0     0    0     1      0      0    0    0
  125    0    0    1     0    0     1      1      0    0    0
  127    0    0    0     0    1     1      0      0    1    0
  133    0    1    1     0    0     0      0      0    0    0
  190    0    0    1     0    0     0      0      0    0    0
  29     0    2    0     0    1     0      0      0    0    0
  534    0    1    0     0    0     1      0      0    0    0
  602    0    1    0     0    0     1      0      0    0    0
  716    0    0    0     1    0     0      0      0    0    0
  863    0    1    0     0    0     0      0      0    0    0


In [111]:
labels=df$Liked
df=as.data.frame(as.matrix(X))
df$Liked=factor(labels,levels=c(0,1))
head(df)

absolut,acknowledg,actual,ago,almost,also,although,alway,amaz,ambianc,⋯,wow,wrap,wrong,year,yet,youd,your,yummi,zero,Liked
0,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


# Train/test

In [112]:
set.seed(123)
split=sample.split(df$Liked, SplitRatio=0.8)
training_set=subset(df,split)
test_set=subset(df,!split)
print(noquote(paste('Training set:',paste(dim(training_set),collapse='x'))))
print(noquote(paste('Test set:',paste(dim(test_set),collapse='x'))))

[1] Training set: 800x692
[1] Test set: 200x692


# Naive Bayes

In [113]:
writeLines('\n=== Naive Bayes ===')
my_classifier=naiveBayes(formula=Liked~.,data=training_set)
writeLines('\nClassifier summary:')
print(summary(my_classifier))
y_pred=predict(my_classifier,newdata=test_set)
writeLines('\nConfusion matrix:')
print(table(test_set$Liked,y_pred))


=== Naive Bayes ===

Classifier summary:
        Length Class  Mode     
apriori   2    table  numeric  
tables  691    -none- list     
levels    2    -none- character
call      4    -none- call     

Confusion matrix:
   y_pred
     0  1
  0  5 95
  1  4 96


# Decision Tree

In [116]:
writeLines('\n=== Decision Tree ===')
my_classifier=rpart(formula=Liked~.,data=training_set,
                    method='class',control=rpart.control(minsplit=1))
writeLines('\nClassifier summary:')
print(summary(my_classifier))
y_pred=predict(my_classifier,newdata=test_set,type='class')
writeLines('\nConfusion matrix:')
print(table(test_set$Liked,y_pred))


=== Decision Tree ===

Classifier summary:
Call:
rpart(formula = Liked ~ ., data = training_set, method = "class", 
    control = rpart.control(minsplit = 1))
  n= 800 

           CP nsplit rel error xerror       xstd
1  0.12750000      0    1.0000 1.0650 0.03528057
2  0.08500000      1    0.8725 0.9125 0.03521973
3  0.04500000      2    0.7875 0.7875 0.03454786
4  0.03750000      4    0.6975 0.7250 0.03399219
5  0.02750000      5    0.6600 0.6750 0.03343604
6  0.02500000      7    0.6050 0.6400 0.03298485
7  0.01833333      8    0.5800 0.6075 0.03251814
8  0.01750000     11    0.5250 0.6050 0.03248028
9  0.01250000     12    0.5075 0.5625 0.03179217
10 0.01000000     13    0.4950 0.5375 0.03134673

Variable importance
    great      good    delici      love      amaz      nice   fantast   perfect 
       21        10         9         8         8         6         6         5 
   awesom      best    friend      vega      feel highlight      wall  eggplant 
        5         5       

# Random Forest, 10 trees

In [122]:
name='Random Forests, 10 trees'
writeLines(paste('\n===',name,'==='))
my_classifier=randomForest(x=training_set[,!(colnames(training_set)=='Liked')],y=training_set$Liked,ntree=10)
writeLines('\nClassifier summary:')
print(summary(my_classifier))
y_pred=predict(my_classifier,newdata=test_set,type='class')
writeLines('\nConfusion matrix:')
print(table(test_set$Liked,y_pred))


=== Random Forests, 10 trees ===

Classifier summary:
                Length Class  Mode     
call               4   -none- call     
type               1   -none- character
predicted        800   factor numeric  
err.rate          30   -none- numeric  
confusion          6   -none- numeric  
votes           1600   matrix numeric  
oob.times        800   -none- numeric  
classes            2   -none- character
importance       691   -none- numeric  
importanceSD       0   -none- NULL     
localImportance    0   -none- NULL     
proximity          0   -none- NULL     
ntree              1   -none- numeric  
mtry               1   -none- numeric  
forest            14   -none- list     
y                800   factor numeric  
test               0   -none- NULL     
inbag              0   -none- NULL     

Confusion matrix:
   y_pred
     0  1
  0 76 24
  1 23 77


# Random Forest, 100 trees

In [123]:
name='Random Forests, 100 trees'
writeLines(paste('\n===',name,'==='))
my_classifier=randomForest(x=training_set[,!(colnames(training_set)=='Liked')],y=training_set$Liked,ntree=100)
writeLines('\nClassifier summary:')
print(summary(my_classifier))
y_pred=predict(my_classifier,newdata=test_set,type='class')
writeLines('\nConfusion matrix:')
print(table(test_set$Liked,y_pred))


=== Random Forests, 100 trees ===

Classifier summary:
                Length Class  Mode     
call               4   -none- call     
type               1   -none- character
predicted        800   factor numeric  
err.rate         300   -none- numeric  
confusion          6   -none- numeric  
votes           1600   matrix numeric  
oob.times        800   -none- numeric  
classes            2   -none- character
importance       691   -none- numeric  
importanceSD       0   -none- NULL     
localImportance    0   -none- NULL     
proximity          0   -none- NULL     
ntree              1   -none- numeric  
mtry               1   -none- numeric  
forest            14   -none- list     
y                800   factor numeric  
test               0   -none- NULL     
inbag              0   -none- NULL     

Confusion matrix:
   y_pred
     0  1
  0 74 26
  1 21 79
